In [1]:
###Reyes, Marcus Group 7

###CoE 197Z Project 1.1

import pandas as pd

import keras
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout,BatchNormalization
from keras import regularizers
from keras.optimizers import adam,sgd
from keras import optimizers

import numpy as np
from numpy import genfromtxt

from sklearn import preprocessing

import matplotlib.pyplot as plt

import string

#from sixfunctions import load_train, load_x_test, clean_data_with_mean
!pip install category_encoders

Using TensorFlow backend.


In [50]:
from importlib import reload
import sixfunctions
sixfunctions = reload(sixfunctions)
from sixfunctions import load_train, load_x_test

In [51]:
#[reason] / dup == duplicate
                  
unique_cols = { 'funder',       # 1898 unique values
                'ward',         # 2092 unique values
                'installer',    # 2146 unique values
                'scheme_name',  # 2697 unique values
                'subvillage',   # 19288 unique values
                'wpt_name' }    # 37400 unique values

do_not_include = {'quality_group',      # exact duplicate of 'quality'
                  'recorded_by',        # uniform value

                  'wpt_name',           #unique
                  'subvillage',         #dup of loc, numerous
                  'scheme_name',        #messy numerous
                  'payment',            #duplicate of payment type
                  'quantity_group',     #Dup of quantity
                  'waterpoint_type_group',#dup of wtptype less data
                  'source_type',        #dup of source
                  'extraction_type',    #dup of extr_type_group
                  #'extraction_type_group',                 
                  'region'} - unique_cols#enc in regcode

#These are sort of ordinal data
do_not_one_hot = {'id','gps_height','doy_recorded_sin','doy_recorded_cos',
                  'longitude','latitude','population','amount_tsh'} | unique_cols

clean_up = {'population','amount_tsh'}

#Tentatively do not include.
do_not_include_tent = {'funder','installer','ward','lga'} - unique_cols

#Don't know what it means 
do_not_include_temp = {'num_private'} - unique_cols


x, train_col,y = load_train("train_set_values.csv","train_set_labels.csv",
                            do_not_include, do_not_one_hot,
                            clean_up, do_not_include_tent, do_not_include_temp,
                            unique_cols)

About to clean up
About to clean up
one-hot encoding:
	basin:	9 categories
	region_code:	27 categories
	district_code:	20 categories
	public_meeting:	2 categories
	scheme_management:	13 categories
	permit:	2 categories
	construction_year:	55 categories
	extraction_type_group:	13 categories
	extraction_type_class:	7 categories
	management:	12 categories
	management_group:	5 categories
	payment_type:	7 categories
	water_quality:	8 categories
	quantity:	5 categories
	source:	10 categories
	source_class:	3 categories
	waterpoint_type:	7 categories
	year_recorded:	5 categories
BINARY ENCODING:
	 subvillage 	 19288  unique values  -> 15  columns
	 funder 	 1898  unique values  -> 11  columns
	 installer 	 2146  unique values  -> 12  columns
	 scheme_name 	 2697  unique values  -> 12  columns
	 ward 	 2092  unique values  -> 12  columns
	 wpt_name 	 37400  unique values  -> 16  columns


In [52]:
###Model

hidden = 1024
dropout = 0.4

(trash, input_dim) = x.shape

activation = 'relu'

model = Sequential()


model.add(Dense(hidden, input_dim = input_dim))
model.add(Dropout(dropout))
model.add(Activation(activation))

model.add(Dense(hidden, input_dim = input_dim))
model.add(Dropout(dropout))
model.add(Activation(activation))

model.add(Dense(hidden, input_dim = input_dim))
model.add(Dropout(dropout))
model.add(Activation(activation))


model.add(Dense(3,input_dim = hidden))
model.add(Activation('softmax'))

model.summary

model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

#history = model.fit(x, y, epochs=20, batch_size=4096*16, validation_split=0.2)
history = model.fit(x, y, epochs=20, batch_size=256, validation_split=0.2)

Train on 47520 samples, validate on 11880 samples
Epoch 1/20
47520/47520 [==============================] - 4s 81us/step - loss: 0.6812 - acc: 0.7136 - val_loss: 0.6059 - val_acc: 0.7572
Epoch 2/20
47520/47520 [==============================] - 3s 60us/step - loss: 0.5914 - acc: 0.7582 - val_loss: 0.5725 - val_acc: 0.7699
Epoch 3/20
47520/47520 [==============================] - 3s 60us/step - loss: 0.5467 - acc: 0.7772 - val_loss: 0.5500 - val_acc: 0.7699
Epoch 4/20
47520/47520 [==============================] - 3s 60us/step - loss: 0.5135 - acc: 0.7905 - val_loss: 0.5491 - val_acc: 0.7756
Epoch 5/20
47520/47520 [==============================] - 3s 60us/step - loss: 0.4874 - acc: 0.7986 - val_loss: 0.5305 - val_acc: 0.7853
Epoch 6/20
47520/47520 [==============================] - 3s 59us/step - loss: 0.4619 - acc: 0.8093 - val_loss: 0.5445 - val_acc: 0.7817
Epoch 7/20
47520/47520 [==============================] - 3s 60us/step - loss: 0.4439 - acc: 0.8158 - val_loss: 0.5228 - val_acc

KeyboardInterrupt: ignored

In [0]:

max_score = -1
iter_of_max = 0,0
train_plot = []
val_plot = []
x_axis = []

#Save the weights for k-fold validation
model.save_weights('model.h5')

amount = 120
k_folds = 5

###When i use k_folds instead of 10 for the np.zeros initialization it acts up
val_plot = np.zeros((k_folds,amount))
train_plot = np.zeros((k_folds,amount))
test_plot = np.zeros((k_folds,amount)) 
x_axis = np.zeros((k_folds,amount))
total = int(x.shape[0])
for j in range(k_folds):

    whole = np.arange(0,total)
    test_range = np.arange((j)*(total/k_folds), (j)*(total/k_folds)+(total/k_folds),dtype = 'int')
    
    train_range = np.delete(whole, test_range)
    
    
    #K-fold validation setup
    x_train = x[train_range,:]
    x_pretest = x[test_range,:]
    y_train = y[train_range,:]
    y_pretest = y[test_range,:]
    
    for i in range(amount):

        history = model.fit(x_train, y_train, epochs = 1, batch_size = 4096*16, verbose = 0)
        score = model.evaluate(x_pretest, y_pretest, batch_size = 512, verbose = 0)

        if  float(100 * score[1]) > float(max_score):
            max_score = float(100 * score[1])
            iter_of_max = i,j
        if i%10 == 0:
            print("----------",i,"-",j,"-----------")
            print("Test accuracy: ", (100.0 * score[1]))
            print("Maxscore: ", max_score, "at", iter_of_max,"epoch-kthfold")
        
        #for the plot
        x_axis[j,i] = i
        test_plot[j,i] = (score[1])
        train_plot[j,i] = np.array(history.history['acc'])

        

    if j == k_folds - 1:
        break
    model.load_weights('model.h5')
    print("Reloading Model")
    
print("Done evaluating performance")